In [1]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from scipy.optimize import minimize_scalar
from sklearn.metrics import mean_squared_error
import time


class RandomForestMSE:
    def __init__(self, n_estimators, max_depth=None, feature_subsample_size=None,
                 object_subsample_size=None,
                 **trees_parameters):
        """
        n_estimators : int
            The number of trees in the forest.

        max_depth : int
            The maximum depth of the tree. If None then there is no limits.

        feature_subsample_size : float
            The size of feature set for each tree. If None then use recommendations.
        """

        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.feature_subsample_size = feature_subsample_size
        self.object_subsample_size = object_subsample_size
        self.trees_parameters = trees_parameters
        
    def fit(self, X, y):
        """
        X : numpy ndarray
            Array of size n_objects, n_features

        y : numpy ndarray
            Array of size n_objects
        """

        if self.feature_subsample_size is None:
            self.feature_subsample_size = X.shape[1] // 3
        if self.object_subsample_size is None:
            self.object_subsample_size = X.shape[0]
        

        indexes_obj_all = np.arange(X.shape[0])
        self.tree_models = []
        for i in range(self.n_estimators):
            indexes_obj_subset = np.random.choice(indexes_obj_all,
                                                  size=self.object_subsample_size,
                                                  replace=True)
            dec_tree = None
            dec_tree = DecisionTreeRegressor(max_depth=self.max_depth,
                                             max_features=self.feature_subsample_size,
                                             **self.trees_parameters)
            dec_tree.fit(X[indexes_obj_subset],
                         y[indexes_obj_subset])
            self.tree_models.append(dec_tree)
            del dec_tree

    def predict(self, X):
        """
        X : numpy ndarray
            Array of size n_objects, n_features

        Returns
        -------
        y : numpy ndarray
            Array of size n_objects
        """

        preds = np.zeros(X.shape[0])
        for i in range(self.n_estimators):
            preds += self.tree_models[i].predict(X) / self.n_estimators
        return preds


class GradientBoostingMSE:
    def __init__(self, n_estimators, learning_rate=0.1, max_depth=5, feature_subsample_size=None,
                 object_subsample_size=None,
                 **trees_parameters):
        """
        n_estimators : int
            The number of trees in the forest.

        learning_rate : float
            Use learning_rate * gamma instead of gamma

        max_depth : int
            The maximum depth of the tree. If None then there is no limits.

        feature_subsample_size : float
            The size of feature set for each tree. If None then use recommendations.
        """

        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        
        self.feature_subsample_size = feature_subsample_size
        self.object_subsample_size = object_subsample_size
        self.trees_parameters = trees_parameters

    def fit(self, X, y):
        """
        X : numpy ndarray
            Array of size n_objects, n_features

        y : numpy ndarray
            Array of size n_objects
        """
        
        if self.feature_subsample_size is None:
            self.feature_subsample_size = X.shape[1] // 3
        if self.object_subsample_size is None:
            self.object_subsample_size = X.shape[0]
        
        
        indexes_obj_all = np.arange(X.shape[0])

        self.obj_indexes = []
        self.alpha_arr = []
        self.models_arr = []

        predict_sum = np.zeros(X.shape[0])

        for i in range(self.n_estimators):
            dec_tree = DecisionTreeRegressor(**self.trees_parameters, max_depth=self.max_depth,
                                             max_features=self.feature_subsample_size)
            indexes_obj_subset = np.random.choice(indexes_obj_all,
                                                  size=self.object_subsample_size,
                                                  replace=True)
            # optimize model

            s_i = 2 * (y[indexes_obj_subset] - predict_sum[indexes_obj_subset])

            dec_tree.fit(X[indexes_obj_subset], s_i)
            pred_opt = dec_tree.predict(X[indexes_obj_subset])
            # optimize model coef

            alpha_i = minimize_scalar(lambda alpha_opt: np.mean((- y[indexes_obj_subset] + predict_sum[indexes_obj_subset] + alpha_opt * pred_opt) ** 2),
                                      bounds=(0, 1000),
                                      method='Bounded').x

            self.obj_indexes.append(indexes_obj_subset)
            
            self.alpha_arr.append(alpha_i * self.learning_rate)
            self.models_arr.append(dec_tree)

            predict_sum += self.models_arr[-1].predict(X[indexes_obj_subset]) * self.alpha_arr[-1]

    def predict(self, X):
        """
        X : numpy ndarray
            Array of size n_objects, n_features

        Returns
        -------
        y : numpy ndarray
            Array of size n_objects
        """

        pred = 0
        preds_all = []
        self.rmse_per_iter = []
        for i in range(len(self.models_arr)):
            preds_all.append(self.models_arr[i].predict(X))
        return np.sum(np.array(self.alpha_arr)[:, None] * np.array(preds_all), axis=0)


    def collect_statistics(self, X, y):

        pred = 0
        preds_all = []
        self.statistics = {'time': [], 'rmse': [], 'max_depth': self.max_depth, 'learning_rate': self.learning_rate,
                           'feature_sub_sample_size': self.feature_subsample_size}
        for i in range(len(self.models_arr)):
            start_time = time.time()
            preds_all.append(self.models_arr[i].predict(X))

            pred = np.sum(np.array(self.alpha_arr)[:i + 1, None] * np.array(preds_all), axis=0)
            self.statistics['rmse'].append(np.sqrt(mean_squared_error(y, pred)))
            self.statistics['time'].append(time.time() - start_time)
        return np.sum(np.array(self.alpha_arr)[:, None] * np.array(preds_all), axis=0)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [25]:
import pandas as pd

train_data = pd.read_csv('./data/data.csv', index_col=0)
target = pd.read_csv('./data/target.csv', index_col=0)

Удалим id, так как он не несет никакую информацию для модели, и преобразуем date с помощью `LabelEncoder`.

In [26]:
from sklearn.preprocessing import LabelEncoder
train_data.drop(columns='id', inplace=True)

le = LabelEncoder()
train_data['date'] = le.fit_transform(train_data['date'].values)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(train_data, target, train_size=0.8,
                                                    random_state=13)

In [6]:
rand_forest_my = RandomForestMSE(n_estimators=100, max_depth=10, feature_subsample_size=None)
rand_forest_sklearn = RandomForestRegressor(n_estimators=100, max_depth=10)

In [7]:
from sklearn.metrics import mean_squared_error

In [8]:
%%time
rand_forest_my.fit(X_train.values, y_train.values)

CPU times: user 2.31 s, sys: 3.03 ms, total: 2.32 s
Wall time: 2.32 s


In [9]:
preds_my = rand_forest_my.predict(X_test.values)

In [10]:
np.sqrt(mean_squared_error(y_test, preds_my))

347312.69204946

In [11]:
%%time
rand_forest_sklearn.fit(X_train.values, y_train.values)

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


CPU times: user 4.81 s, sys: 7.77 ms, total: 4.81 s
Wall time: 4.81 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [12]:
preds_sklearn = rand_forest_sklearn.predict(X_test.values)

In [13]:
np.sqrt(mean_squared_error(y_test, preds_sklearn))

347558.0570251268

In [14]:
gb_my = GradientBoostingMSE(n_estimators=100, max_depth=3, feature_subsample_size=None, learning_rate=0.1)
# rand_forest_sklearn = RandomForestRegressor(n_estimators=100, max_depth=10)

In [15]:
%%time

gb_my.fit(X_train.values, y_train.values.reshape(-1))

CPU times: user 12.9 s, sys: 304 ms, total: 13.2 s
Wall time: 1.24 s


In [16]:
gb_my_preds = gb_my.predict(X_test.values)

In [17]:
np.sqrt(mean_squared_error(y_test, gb_my_preds))

354685.63164512755

In [18]:
from sklearn.ensemble import GradientBoostingRegressor

gb_sklearn = GradientBoostingRegressor(criterion='mse',
                                       n_estimators=1000, max_depth=3, max_features=None, learning_rate=0.1)

In [19]:
%%time

gb_sklearn.fit(X_train.values, y_train.values.reshape(-1))

CPU times: user 1min 18s, sys: 1.85 s, total: 1min 20s
Wall time: 7.31 s


GradientBoostingRegressor(alpha=0.9, criterion='mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [20]:
gb_sklearn_preds = gb_sklearn.predict(X_test.values)

In [21]:
np.sqrt(mean_squared_error(y_test, gb_sklearn_preds))

364261.47239136806

In [22]:
gb_my.collect_statistics(X_test.values, y_test.values.reshape(-1))

array([586599.06067645, 510982.07223391, 623026.41954241, ...,
       524063.33436287, 549963.79467707, 592287.04645928])

In [23]:
np.cumsum(gb_my.statistics['time'])

array([0.00040317, 0.00071025, 0.00099778, 0.00131035, 0.00160933,
       0.00193501, 0.00224257, 0.00256443, 0.00288272, 0.00321269,
       0.00354004, 0.00387001, 0.00420189, 0.00458884, 0.00494671,
       0.00532269, 0.00569463, 0.00608087, 0.00761986, 0.00813603,
       0.00861192, 0.00927758, 0.00990868, 0.01037216, 0.01102638,
       0.01151371, 0.01198649, 0.01249647, 0.01298285, 0.01345873,
       0.01398253, 0.01448345, 0.01498485, 0.01553774, 0.01609039,
       0.01663017, 0.01722026, 0.01775503, 0.0182972 , 0.01881313,
       0.0193584 , 0.01991034, 0.02047133, 0.0209043 , 0.02128696,
       0.02166581, 0.0220716 , 0.02246237, 0.02285504, 0.02328038,
       0.02368164, 0.02408266, 0.02450395, 0.02492404, 0.02535963,
       0.02577543, 0.02621055, 0.02664518, 0.02720952, 0.02788162,
       0.02856445, 0.02923203, 0.02988267, 0.03058386, 0.03126597,
       0.03173971, 0.03220582, 0.03272295, 0.03319883, 0.03369403,
       0.03418517, 0.03468776, 0.03521752, 0.0357337 , 0.03627